In [4]:
!pip install -q transformers datasets accelerate bitsandbytes wandb peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:

import huggingface_hub
huggingface_hub.login()


In [5]:
import os
import time
import wandb
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig
from peft import LoraConfig, get_peft_model, LoraConfig, TaskType

# wandb 초기화
wandb.init(project="Hanghae99_week8_advanced", name="gpt2_plain_experiment")

data_path = "corpus.json"

with open(data_path, "r", encoding="utf-8") as f:
    corpus = json.load(f)

dataset = Dataset.from_dict({
    "question": [item["question"] for item in corpus],
    "answer": [item["answer"] for item in corpus]
})

train_split = dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
train_dataset = train_split["train"]
eval_dataset = train_split["test"]

# ===== 모델과 토크나이저 로드 (경량화 기법 없이) =====
MODEL_NAME = "openai-community/gpt2"
print(f"Loading model {MODEL_NAME} normally (without lightweight techniques)...")
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = 1024

# 모델 파라미터 수 로깅
n_params = sum(p.numel() for p in model.parameters())
wandb.log({f"{MODEL_NAME}_num_params": n_params})
print(f"{MODEL_NAME} parameters: {n_params}")

# ===== 평가용 함수 정의 =====
def evaluate_inference_speed(model, tokenizer, prompt, num_runs=3):
    device = 0 if torch.cuda.is_available() else -1
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)
    times = []
    for i in range(num_runs):
        start = time.time()
        _ = generator(prompt, max_length=100)
        elapsed = time.time() - start
        times.append(elapsed)
        print(f"Run {i+1}: {elapsed:.3f} sec")
    avg_time = sum(times) / len(times)
    return avg_time

# ===== 미세조정 전 평가 (Pre fine-tuning evaluation) =====
instruction = "How does the VIX (Volatility Index) influence trading decisions in the stock market"

print("\n[Pre fine-tuning evaluation]")

# 정량적 평가: 추론 속도 측정
avg_time = evaluate_inference_speed(model, tokenizer, instruction)
wandb.log({f"{MODEL_NAME}_avg_inference_time_pre": avg_time})
print(f"Avg Inference Time (pre): {avg_time:.3f} sec")

# 정성적 평가: 동일 instruction에 대한 응답 생성
device = 0 if torch.cuda.is_available() else -1
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)
output = generator(instruction, max_length=250)
response_text = output[0]['generated_text']
wandb.log({f"{MODEL_NAME}_response_pre": response_text})
print(f"Response (pre):\n{response_text}")
print("-" * 60)

# ===== 미세조정(Fine-tuning) 준비 =====
def formatting_prompts_func(data):
    output_texts = []
    for i in range(len(data['question'])):
        text = f"### Question: {data['question'][i]}\n ### Answer: {data['answer'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# ===== Fine-tuning 설정 및 실행 =====
training_config = SFTConfig(
    output_dir="./gpt2_2b_lt_output",
    save_total_limit=1,
    logging_steps=100,
    eval_steps=100,
    max_steps=1000,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_steps=100,
    save_strategy="steps",
    logging_strategy="steps",
    evaluation_strategy="steps",
)

trainer = SFTTrainer(
    model=model,
    args=training_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

print(f"\nStarting fine-tuning for {MODEL_NAME} ...")
train_start_time = time.time()
trainer.train()
train_end_time = time.time()
total_training_time = (train_end_time - train_start_time) / 60
wandb.log({"train/total_training_time_min": total_training_time})
print(f"Fine-tuning for {MODEL_NAME} completed.")

# ===== 7. 미세조정 후 평가 (Post fine-tuning evaluation) =====
print("\n[Post fine-tuning evaluation]")

avg_time_post = evaluate_inference_speed(model, tokenizer, instruction)
wandb.log({f"{MODEL_NAME}_avg_inference_time_post": avg_time_post})
print(f"Avg Inference Time (post): {avg_time_post:.3f} sec")

# 정성 평가: 동일 instruction에 대한 응답 생성
output_post = generator(instruction, max_length=250)
response_text_post = output_post[0]['generated_text']
wandb.log({f"{MODEL_NAME}_response_post": response_text_post})
print(f"Response (post):\n{response_text_post}")
print("-" * 60)

# ===== 8. 마무리 =====
wandb.finish()


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: code-sugartoast (code-sugartoast-sugartoast) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Loading model openai-community/gpt2 normally (without lightweight techniques)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
100,2.595400,2.272718
200,2.366500,2.161808
300,2.254900,2.093556
400,2.197100,2.044530
500,2.174800,2.013694
600,2.141000,1.984660
700,2.115700,1.966273
800,2.048500,1.951562
900,2.045900,1.941659
1000,2.097100,1.938540


Device set to use cuda:0


Fine-tuning for openai-community/gpt2 completed.

[Post fine-tuning evaluation]
Run 1: 0.848 sec
Run 2: 0.841 sec
Run 3: 0.834 sec
Avg Inference Time (post): 0.841 sec
Response (post):
How does the VIX (Volatility Index) influence trading decisions in the stock market? During early trading volumes, investors can focus on asset performance, historical performance, and overall market conditions that predict the future price of VIX stocks. The VIX is driven by market optimism and sentiment, which can influence the trading decisions of trading traders.

VIX Traders often assess the impact of economic conditions, market sentiment, and economic forecasts. Factors such as GDP growth, the participation of large multinational corporations, and changing investor expectations can influence stock prices.

Historical Trends and Sentiment Factors

Historical trends and sentiment factors are crucial for trading decisions as they predict market trends, market sentiment—and consumer opinions—based on f

eval/loss,█▆▄▃▃▂▂▁▁▁
eval/mean_token_accuracy,▁▃▅▆▆▇▇███
eval/runtime,▃▁▁▂▁█▂▃▃▁
eval/samples_per_second,▆██▇█▁▇▆▆█
eval/steps_per_second,▆██▇█▁▇▆▆█
openai-community/gpt2_avg_inference_time_post,▁
openai-community/gpt2_avg_inference_time_pre,▁
openai-community/gpt2_num_params,▁
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██████
train/grad_norm,▁▅▃▃▇▃▁▂██
